In [1]:
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')
cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )
   

In [129]:
cur = cnx.cursor()

In [11]:
recados=pd.read_sql("SELECT * FROM recados",cnx)
egresados=pd.read_sql("SELECT * FROM egresados",cnx)
carreras=pd.read_sql("SELECT * FROM carreras",cnx)
usuarios=pd.read_sql("SELECT * FROM users",cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

In [81]:
muestras=pd.read_sql("SELECT * FROM muestras where estudio_id='3'",cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [2]:
egresados=pd.read_sql("SELECT * FROM egresados",cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
recados_merge=recados.merge(egresados[['cuenta','carrera','plantel']],how='inner',on='cuenta')

In [130]:
#Establecer umbral de vuelta (cuanto se entiende que es el intervalo de descanso minimo)
tresh=19 #22 dias naturales¿?
hoy=datetime.datetime.now()

In [18]:
#filtrar recados
carrera='101'
plantel='3'
rec=recados_merge.loc[(recados_merge['carrera']==carrera)&(recados_merge['plantel']==plantel)]
print(len(rec))

118


In [84]:
655/2

327.5

In [34]:
import datetime

In [ ]:
ordered_rec=rec.sort_values(by='fecha',ascending=True)
#dar formato a las fechas ordenadas
ordered_rec['fecha']=ordered_rec['fecha'].astype(str).apply(lambda x: x[:10])

In [134]:
for k in range(len(muestras)):
    #filtrar recados
    carrera=str(muestras['carrera_id'].values[k])
    plantel=str(muestras['plantel_id'].values[k])
    rec=recados_merge.loc[(recados_merge['carrera']==carrera)&(recados_merge['plantel']==plantel)]
    print('carrera',carrera,plantel,len(rec))
    fecha_inicial=datetime.datetime.strptime(str(ordered_rec['fecha'].values[0]),'%Y-%m-%d')
    nvuelta=1
    print(fecha_inicial)
    for i in range(len(ordered_rec)):
        this_fecha=datetime.datetime.strptime(str(ordered_rec['fecha'].values[i]),'%Y-%m-%d')
        distance=this_fecha-fecha_inicial
        if(distance.days>tresh):
            print('esta ya viene a ser otra vuelta',i,this_fecha)
            #Ingresar fehca inicio de la vuelta a la base
            cur.execute("INSERT INTO vueltas(muestra_id,inicio,numero_vuelta,created_at) VALUES (%s, %s,%s,%s)", (str(muestras['id'].values[k]),str(fecha_inicial),str(nvuelta),str(hoy))) 
            fecha_inicial=this_fecha
            nvuelta=nvuelta+1
        # print(distance,ordered_rec['fecha'].values[i])
    cur.execute("INSERT INTO vueltas(muestra_id,inicio,numero_vuelta,created_at) VALUES (%s, %s,%s,%s)", (str(muestras['id'].values[k]),fecha_inicial,str(nvuelta),hoy)) 
            

carrera 102 1 701
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra vuelta 72 2024-09-23 00:00:00
carrera 103 1 19
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra vuelta 72 2024-09-23 00:00:00
carrera 105 1 75
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra vuelta 72 2024-09-23 00:00:00
carrera 123 1 58
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra vuelta 72 2024-09-23 00:00:00
carrera 401 2 310
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra vuelta 72 2024-09-23 00:00:00
carrera 406 2 5
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra vuelta 72 2024-09-23 00:00:00
carrera 423 2 412
2024-04-22 00:00:00
esta ya viene a ser otra vuelta 71 2024-05-14 00:00:00
esta ya viene a ser otra

In [135]:
vueltas=pd.read_sql('select * from vueltas',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [136]:
vueltas['numero_vuelta'].unique()

array(['1', '2', '3'], dtype=object)

In [137]:
cnx.commit()

In [127]:
cur.close()
cnx.close()

In [52]:
datetime.datetime.strptime('Jun 1 2005', '%b %d %Y').date()

datetime.date(2005, 6, 1)

In [82]:
len(muestras)

176

In [83]:
muestras

,id,estudio_id,carrera_id,plantel_id,enc_id,requeridas_10,requeridas_5,dispersion,poblacion
0,524,3,102,1,None,33,59,None,788
1,525,3,103,1,None,12,13,None,14
2,526,3,105,1,None,22,24,None,35
3,527,3,123,1,None,20,20,None,28
4,528,3,401,2,None,59,56,None,194
...,...,...,...,...,...,...,...,...,...
171,695,3,434,700,None,16,17,None,20
172,696,3,437,700,None,1,1,None,1
173,697,3,438,700,None,5,6,None,6
174,698,3,501,700,None,1,1,None,1


In [4]:
reactivos=pd.read_sql('select * from reactivos',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
for i in range(len(reactivos)):
    print(reactivos['clave'].values[i],reactivos['description'].values[i],reactivos['type'].values[i],)

nar8 ¿Cuál es su estado civil? option
nar9 ¿Tiene hijos? option
nar10 ¿Cuántos? number
nar11 ¿Cuál es el último grado de estudios de su pareja? option
nar11a Otro: text
nar14 ¿Cuál es la ocupación de su pareja? option
nar14otra Otro: text
nar12 En el transcurso de su carrera profesional, ¿cuál era el nivel máximo de estudios de su madre? option
nar12a Otro: text
nrx Si su madre es profesionista, ¿cursó sus estudios en la UNAM? option
nar15 En el transcurso de su carrera profesional, ¿cuál era la ocupación de su madre? option
nar13 En el transcurso de su carrera profesional, ¿cuál era el nivel máximo de estudios de su padre? option
nar13a Otro: text
nrxx Si su padre es profesionista, ¿cursó sus estudios en la UNAM? option
nar16 En el transcurso de su carrera profesional, ¿cuál era la ocupación de su padre? option
nar16otra Otro: text
nar15otra Otro: text
nbr1 ¿Qué tipo de bachillerato cursó? option
ner20 ¿Tiene una segunda licenciatura? option
ner20txt ¿Cuál? text
ner20a ¿La ejerce? opt

In [8]:
reactivos.orden.unique()

array([ 1.1 ,  2.  ,  2.1 ,  3.  ,  3.1 ,  4.  ,  4.1 ,  5.  ,  5.1 ,
        5.2 ,  6.  ,  7.  ,  7.1 ,  7.2 ,  8.  ,  8.1 ,  6.1 ,  9.  ,
       10.  , 10.1 , 10.2 ,  9.1 , 12.  , 13.  , 14.  , 15.  , 16.  ,
       17.  , 35.  , 23.1 , 17.21, 18.  , 19.  , 20.  , 21.  , 22.  ,
       23.  , 24.  , 25.  , 26.  , 27.  , 28.  , 29.  , 30.  , 31.  ,
       32.  , 33.  , 34.  , 36.  , 37.  , 38.  , 39.  , 40.  , 41.  ,
       17.2 , 42.  , 43.  , 44.  , 45.  , 46.  , 47.  , 48.  , 49.  ,
       50.  , 51.  , 52.  , 53.  , 54.  , 55.  , 57.  , 58.  , 59.  ,
       60.  , 61.  , 66.  , 67.  , 68.  , 69.  , 70.  , 71.  ,  1.  ,
       11.  , 56.  , 62.  , 63.  , 64.  , 65.  , 54.5 ,   nan,  3.5 ,
       32.5 , 63.1 , 63.2 , 63.3 , 63.4 , 22.1 , 28.5 ])